In [1]:
import pandas as pd

df = pd.read_csv("fish_all.csv")

In [2]:
df.head()

,Species,Weight,Length1,Length2,Length3,Height,Width
0,Bream,242.0,23.2,25.4,30.0,11.5200,4.0200
1,Bream,290.0,24.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,23.9,26.5,31.1,12.3778,4.6961
3,Bream,363.0,26.3,29.0,33.5,12.7300,4.4555
4,Bream,430.0,26.5,29.0,34.0,12.4440,5.1340


In [3]:
df['Species'].value_counts()

Perch        56
Bream        35
Roach        20
Pike         17
Smelt        14
Parkki       11
Whitefish     6
Name: Species, dtype: int64

In [4]:
df.loc[df['Species']=='Perch','Species'] = 0
df.loc[df['Species']=='Bream','Species'] = 1
df.loc[df['Species']=='Roach','Species'] = 2
df.loc[df['Species']=='Pike','Species'] = 3
df.loc[df['Species']=='Smelt','Species'] = 4
df.loc[df['Species']=='Parkki','Species'] = 5
df.loc[df['Species']=='Whitefish','Species'] = 6
df['Species'] = df['Species'].astype(int)

In [5]:
feature = df[df.keys().drop('Species')]
outcome = df['Species']

In [6]:
from sklearn.model_selection import train_test_split

train_feature, test_featue, train_target, test_target = train_test_split(feature,outcome,random_state=42, stratify=outcome)
train_feature, val_featue, train_target, val_target = train_test_split(train_feature,train_target,random_state=42, stratify=train_target)

In [7]:
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(train_feature, train_target)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [8]:
y_pred = clf.predict(test_featue)

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.675
[[0 0 0 0 0 0 1]
 [0 1 0 0 2 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 3 0 0 1]
 [0 0 0 0 1 0 4]
 [0 0 0 0 0 9 0]
 [0 0 0 2 3 0 9]]


In [10]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42, n_jobs=-1, k_neighbors=3)
train_feature_oversampling, train_target_oversampling = sm.fit_resample(train_feature, train_target)

In [11]:
from xgboost import XGBClassifier

clf2 = XGBClassifier()
clf2.fit(train_feature_oversampling, train_target_oversampling)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [12]:
y_pred = clf2.predict(test_featue)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.625
[[0 0 0 0 0 0 1]
 [0 1 0 0 2 0 0]
 [0 0 4 0 0 0 0]
 [0 0 0 3 0 0 1]
 [1 0 0 0 1 0 3]
 [0 0 0 0 0 9 0]
 [3 0 0 0 3 1 7]]


In [14]:
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

clf3 = XGBClassifier()
early_stop = EarlyStopping(rounds=30,metric_name='merror',data_name="validation_0",save_best=True)
clf3.fit(train_feature, train_target, eval_set=[(val_featue,val_target)], eval_metric='merror',callbacks=[early_stop])

/home/minsuchae/.pyenv/versions/3.9.0/envs/tensorflow_gpu_2.7.0/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/minsuchae/.pyenv/versions/3.9.0/envs/tensorflow_gpu_2.7.0/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-merror:0.43333
[1]	validation_0-merror:0.33333
[2]	validation_0-merror:0.30000
[3]	validation_0-merror:0.23333
[4]	validation_0-merror:0.26667
[5]	validation_0-merror:0.26667
[6]	validation_0-merror:0.26667
[7]	validation_0-merror:0.26667
[8]	validation_0-merror:0.26667
[9]	validation_0-merror:0.26667
[10]	validation_0-merror:0.26667
[11]	validation_0-merror:0.26667
[12]	validation_0-merror:0.26667
[13]	validation_0-merror:0.30000
[14]	validation_0-merror:0.30000
[15]	validation_0-merror:0.30000
[16]	validation_0-merror:0.30000
[17]	validation_0-merror:0.30000
[18]	validation_0-merror:0.30000
[19]	validation_0-merror:0.30000
[20]	validation_0-merror:0.30000
[21]	validation_0-merror:0.30000
[22]	validation_0-merror:0.30000
[23]	validation_0-merror:0.30000
[24]	validation_0-merror:0.30000
[25]	validation_0-merror:0.30000
[26]	validation_0-merror:0.30000
[27]	validation_0-merror:0.30000
[28]	validation_0-merror:0.30000
[29]	validation_0-merror:0.30000
[30]	validation_0-me

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [15]:
y_pred = clf3.predict(test_featue)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.7
[[ 0  0  0  0  0  0  1]
 [ 0  1  0  0  2  0  0]
 [ 0  0  4  0  0  0  0]
 [ 0  0  0  3  0  0  1]
 [ 0  0  0  0  1  0  4]
 [ 0  0  0  0  0  9  0]
 [ 0  0  0  0  4  0 10]]


In [17]:
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

clf4 = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
early_stop = EarlyStopping(rounds=30,metric_name='merror',data_name="validation_0",save_best=True)
clf4.fit(train_feature, train_target, eval_set=[(val_featue,val_target)], eval_metric='merror',callbacks=[early_stop])

/home/minsuchae/.pyenv/versions/3.9.0/envs/tensorflow_gpu_2.7.0/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/home/minsuchae/.pyenv/versions/3.9.0/envs/tensorflow_gpu_2.7.0/lib/python3.9/site-packages/xgboost/sklearn.py:861: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-merror:0.53333
[1]	validation_0-merror:0.40000
[2]	validation_0-merror:0.36667
[3]	validation_0-merror:0.30000
[4]	validation_0-merror:0.30000
[5]	validation_0-merror:0.30000
[6]	validation_0-merror:0.30000
[7]	validation_0-merror:0.30000
[8]	validation_0-merror:0.30000
[9]	validation_0-merror:0.30000
[10]	validation_0-merror:0.30000
[11]	validation_0-merror:0.30000
[12]	validation_0-merror:0.26667
[13]	validation_0-merror:0.26667
[14]	validation_0-merror:0.26667
[15]	validation_0-merror:0.26667
[16]	validation_0-merror:0.26667
[17]	validation_0-merror:0.26667
[18]	validation_0-merror:0.26667
[19]	validation_0-merror:0.26667
[20]	validation_0-merror:0.26667
[21]	validation_0-merror:0.26667
[22]	validation_0-merror:0.26667
[23]	validation_0-merror:0.26667
[24]	validation_0-merror:0.26667
[25]	validation_0-merror:0.26667
[26]	validation_0-merror:0.26667
[27]	validation_0-merror:0.26667
[28]	validation_0-merror:0.26667
[29]	validation_0-merror:0.26667
[30]	validation_0-me

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', ...)

In [18]:
y_pred = clf4.predict(test_featue)

In [19]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.65
[[0 0 0 0 0 0 1]
 [0 1 0 0 2 0 0]
 [0 0 3 0 0 0 1]
 [0 0 0 3 0 0 1]
 [0 0 0 0 2 0 3]
 [0 0 0 0 0 9 0]
 [0 0 0 3 3 0 8]]


In [20]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping
import numpy as np

clf5 = XGBClassifier(tree_method='gpu_hist', gpu_id=0)
parameters = {'gamma':np.arange(0.1,2.1,0.1),'max_depth':np.arange(1,3)}
grid_search = GridSearchCV(clf5, param_grid=parameters, cv=5, n_jobs=-1,scoring='accuracy')

In [21]:
grid_search.fit(train_feature, train_target)

/home/minsuchae/.pyenv/versions/3.9.0/envs/tensorflow_gpu_2.7.0/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None, gpu_id=0,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, 

In [22]:
grid_search.best_params_

{'gamma': 0.1, 'max_depth': 1}

In [23]:
clf5 = grid_search.best_estimator_

In [24]:
y_pred = clf5.predict(test_featue)

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

print("Accuracy :",accuracy_score(test_target, y_pred))
print(confusion_matrix(test_target, y_pred,labels=[6,5,4,3,2,1,0]))

Accuracy : 0.725
[[ 0  0  0  0  1  0  0]
 [ 0  1  0  0  2  0  0]
 [ 0  0  4  0  0  0  0]
 [ 0  0  0  3  0  0  1]
 [ 0  0  0  0  2  0  3]
 [ 0  0  0  0  0  9  0]
 [ 0  0  0  1  3  0 10]]
